# Self-classifying Neural Cellular Automata

This notebook contains code to reproduce experiments and figures for the "Self-classifying Neural Cellular Automata" article [TODO link].

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
#@title imports and notebook utils
%tensorflow_version 2.x

import os
import io
import PIL.Image, PIL.ImageDraw
import base64
import zipfile
import json
import requests
import numpy as np
import matplotlib.pylab as pl
import glob

import tensorflow as tf

from IPython.display import Image, HTML, clear_output
import tqdm

import os
os.environ['FFMPEG_BINARY'] = 'ffmpeg'
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
clear_output()

def np2pil(a):
  if a.dtype in [np.float32, np.float64]:
    a = np.uint8(np.clip(a, 0, 1)*255)
  return PIL.Image.fromarray(a)

def imwrite(f, a, fmt=None):
  a = np.asarray(a)
  if isinstance(f, str):
    fmt = f.rsplit('.', 1)[-1].lower()
    if fmt == 'jpg':
      fmt = 'jpeg'
    f = open(f, 'wb')
  np2pil(a).save(f, fmt, quality=95)

def imencode(a, fmt='jpeg'):
  a = np.asarray(a)
  if len(a.shape) == 3 and a.shape[-1] == 4:
    fmt = 'png'
  f = io.BytesIO()
  imwrite(f, a, fmt)
  return f.getvalue()

def im2url(a, fmt='jpeg'):
  encoded = imencode(a, fmt)
  base64_byte_string = base64.b64encode(encoded).decode('ascii')
  return 'data:image/' + fmt.upper() + ';base64,' + base64_byte_string

def imshow(a, fmt='jpeg'):
  display(Image(data=imencode(a, fmt)))

def tile2d(a, w=None):
  a = np.asarray(a)
  if w is None:
    w = int(np.ceil(np.sqrt(len(a))))
  th, tw = a.shape[1:3]
  pad = (w-len(a))%w
  a = np.pad(a, [(0, pad)]+[(0, 0)]*(a.ndim-1), 'constant')
  h = len(a)//w
  a = a.reshape([h, w]+list(a.shape[1:]))
  a = np.rollaxis(a, 2, 1).reshape([th*h, tw*w]+list(a.shape[4:]))
  return a

def zoom(img, scale=4):
  img = np.repeat(img, scale, 0)
  img = np.repeat(img, scale, 1)
  return img

class VideoWriter:
  def __init__(self, filename, fps=30.0, **kw):
    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()

#Load data

In [ ]:
# @title Get train/test

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.array(x_train / 255.0,).astype(np.float32)
x_test = np.array(x_test / 255.0,).astype(np.float32)

# @title Data generator


color_lookup = tf.constant([
            [128, 0, 0],
            [230, 25, 75],
            [70, 240, 240],
            [210, 245, 60],
            [250, 190, 190],
            [170, 110, 40],
            [170, 255, 195],
            [165, 163, 159],
            [0, 128, 128],
            [128, 128, 0],
            [0, 0, 0], # This is the default for digits.
            [255, 255, 255] # This is the background.
            ])

backgroundWhite = True
def color_labels(x, y_pic, disable_black=False, dtype=tf.uint8):
  # works for shapes of x [b, r, c] and [r, c]
  black_and_white = tf.fill(list(x.shape) + [2], 0.01)
  is_gray = tf.cast(x > 0.1, tf.float32)
  is_not_gray = 1. - is_gray

  y_pic = y_pic * tf.expand_dims(is_gray, -1) # forcibly cancels everything outside of it.
  
  # if disable_black, make is_gray super low.
  if disable_black:
    is_gray *= -1e5
    # this ensures that you don't draw white in the digits.
    is_not_gray += is_gray

  bnw_order = [is_gray, is_not_gray] if backgroundWhite else [is_not_gray, is_gray]
  black_and_white *= tf.stack(bnw_order, -1)

  rgb = tf.gather(
      color_lookup,
      tf.argmax(tf.concat([y_pic, black_and_white], -1), -1))
  if dtype == tf.uint8:
    return tf.cast(rgb, tf.uint8)
  else:
    return tf.cast(rgb, dtype) / 255.

def to_ten_dim_label(x, y):
  # x shape is [b, r, c]
  # y shape is [b]

  # y_res shape is [b, r, c, 10]
  y_res = np.zeros(list(x.shape) + [10])
  # broadcast y to match x shape:
  y_expanded = np.broadcast_to(y, x.T.shape).T
  y_res[x >= 0.1, y_expanded[x >= 0.1]] = 1.0
  return y_res.astype(np.float32)

def find_different_numbers(x_set, y_set, y_set_pic, orientation="vertical"):
  result_y = []
  result_x = []
  for i in range(10):
    for x, y, y_pic in zip(x_set, y_set, y_set_pic):
      if y == i:
        result_y.append(color_labels(x, y_pic))
        result_x.append(x)
        break
  assert len(result_y) == 10

  result_y = np.concatenate(result_y, axis=0 if orientation == "vertical" else 1)
  result_x = np.stack(result_x)

  return result_y, result_x


print("Generating y pics...")
y_train_pic = to_ten_dim_label(x_train, y_train)
y_test_pic = to_ten_dim_label(x_test, y_test)


numbers_legend, x_legend = find_different_numbers(x_train, y_train, y_train_pic)

imshow(zoom(numbers_legend))
print("Storing x_legend for use in the demo.")
pl.imshow(x_legend.reshape((-1, 28)))
import json
samples_str = json.dumps(x_legend.tolist())


## Cellular Automata parameters

In [ ]:
CHANNEL_N = 19        # Number of CA state channels
BATCH_SIZE = 16
POOL_SIZE = BATCH_SIZE * 10
CELL_FIRE_RATE = 0.5


model_type = '3 mutating'  #@param ['1 naive', '2 persistent', '3 mutating']
LOSS_TYPE = "l2"  #@param ['l2', 'ce']
ADD_NOISE = "True"  #@param ['True', 'False']

USE_PATTERN_POOL, MUTATE_POOL = {
    '1 naive': (False, False),
    '2 persistent': (True, False),
    '3 mutating': (True, True)
    }[model_type]
ADD_NOISE = ADD_NOISE == 'True'

In [ ]:
#@title CA Model and utils

from tensorflow.keras.layers import Conv2D

class CAModel(tf.keras.Model):

  def __init__(self, channel_n=CHANNEL_N, fire_rate=CELL_FIRE_RATE,
               add_noise=ADD_NOISE):
    # CHANNEL_N does *not* include the Gray channel.
    # it does include the 10 possible outputs, though.
    super().__init__()
    self.channel_n = channel_n
    self.fire_rate = fire_rate
    self.add_noise = add_noise

    self.perceive = tf.keras.Sequential([
          Conv2D(80, 3, activation=tf.nn.relu, padding="SAME"),
      ])

    self.dmodel = tf.keras.Sequential([
          Conv2D(80, 1, activation=tf.nn.relu),
          Conv2D(self.channel_n, 1, activation=None,
                       kernel_initializer=tf.zeros_initializer),
    ])

    self(tf.zeros([1, 3, 3, channel_n + 1]))  # dummy calls to build the model

  @tf.function
  def call(self, x, fire_rate=None, manual_noise=None):
    gray, state = tf.split(x, [1, self.channel_n], -1)
    ds = self.dmodel(self.perceive(x))
    if self.add_noise:
      if manual_noise is None:
        residual_noise = tf.random.normal(tf.shape(ds), 0., 0.02)
      else:
        residual_noise = manual_noise
      ds += residual_noise

    if fire_rate is None:
      fire_rate = self.fire_rate
    update_mask = tf.random.uniform(tf.shape(x[:, :, :, :1])) <= fire_rate
    living_mask = gray > 0.1
    residual_mask = update_mask & living_mask
    ds *= tf.cast(residual_mask, tf.float32)
    state += ds
    
    return tf.concat([gray, state], -1)

  @tf.function
  def initialize(self, images):
    state = tf.zeros([tf.shape(images)[0], 28, 28, self.channel_n])
    images = tf.reshape(images, [-1, 28, 28, 1])
    return tf.concat([images, state], -1)

  @tf.function
  def classify(self, x):
    # We'll assume that the last 10 layers are the classification.
    # keep in mind that in this version there is no "background" class,
    # and that any loss doesn't propagate to "dead" pixels.
    return x[:,:,:,-10:]

CAModel().perceive.summary()
CAModel().dmodel.summary()

# Training and visualization utils

In [ ]:
#@title Train utils (SamplePool, model export, visualizations)
from google.protobuf.json_format import MessageToDict
from tensorflow.python.framework import convert_to_constants

class SamplePool:
  def __init__(self, *, _parent=None, _parent_idx=None, **slots):
    self._parent = _parent
    self._parent_idx = _parent_idx
    self._slot_names = slots.keys()
    self._size = None 
    for k, v in slots.items():
      if self._size is None:
        self._size = len(v)
      assert self._size == len(v)
      setattr(self, k, np.asarray(v))

  def sample(self, n):
    idx = np.random.choice(self._size, n, False)
    batch = {k: getattr(self, k)[idx] for k in self._slot_names}
    batch = SamplePool(**batch, _parent=self, _parent_idx=idx)
    return batch

  def commit(self):
    for k in self._slot_names:
      getattr(self._parent, k)[self._parent_idx] = getattr(self, k)

def export_model(ca, base_fn):
  ca.save_weights(base_fn)

  cf = ca.call.get_concrete_function(
      x=tf.TensorSpec([None, None, None, CHANNEL_N+1]),
      fire_rate=tf.constant(0.5),
      manual_noise=tf.TensorSpec([None, None, None, CHANNEL_N]))
  cf = convert_to_constants.convert_variables_to_constants_v2(cf)
  graph_def = cf.graph.as_graph_def()
  graph_json = MessageToDict(graph_def)
  graph_json['versions'] = dict(producer='1.14', minConsumer='1.14')
  model_json = {
      'format': 'graph-model',
      'modelTopology': graph_json,
      'weightsManifest': [],
  }
  with open(base_fn+'.json', 'w') as f:
    json.dump(model_json, f)

def classify_and_color(ca, x, disable_black=False):
  return color_labels(
      x[:,:,:,0], ca.classify(x), disable_black, dtype=tf.float32)


def generate_tiled_figures(figures, fade_by=0.1):
  tiled_pool = tile2d(figures)
  fade_sz = int(tiled_pool.shape[0] * fade_by)
  fade = np.linspace(1.0, 0.0, fade_sz)
  ones = np.ones(fade_sz) 
  tiled_pool[:, :fade_sz] += (-tiled_pool[:, :fade_sz] + ones[None, :, None]) * fade[None, :, None] 
  tiled_pool[:, -fade_sz:] += (-tiled_pool[:, -fade_sz:] + ones[None, :, None]) * fade[None, ::-1, None]
  tiled_pool[:fade_sz, :] += (-tiled_pool[:fade_sz, :] + ones[:, None, None]) * fade[:, None, None]
  tiled_pool[-fade_sz:, :] += (-tiled_pool[-fade_sz:, :] + ones[:, None, None]) * fade[::-1, None, None]
  return tiled_pool


def generate_pool_figures(ca, pool, step_i):
  tiled_pool = tile2d(classify_and_color(ca, pool.x))
  fade = np.linspace(1.0, 0.0, 72)
  ones = np.ones(72) 
  tiled_pool[:, :72] += (-tiled_pool[:, :72] + ones[None, :, None]) * fade[None, :, None] 
  tiled_pool[:, -72:] += (-tiled_pool[:, -72:] + ones[None, :, None]) * fade[None, ::-1, None]
  tiled_pool[:72, :] += (-tiled_pool[:72, :] + ones[:, None, None]) * fade[:, None, None]
  tiled_pool[-72:, :] += (-tiled_pool[-72:, :] + ones[:, None, None]) * fade[::-1, None, None]
  imwrite('train_log/%04d_pool.jpg'%step_i, tiled_pool)

def visualize_batch(ca, x0, x, step_i):
  vis0 = np.hstack(classify_and_color(ca, x0).numpy())
  vis1 = np.hstack(classify_and_color(ca, x).numpy())
  vis = np.vstack([vis0, vis1])
  imwrite('train_log/batches_%04d.jpg'%step_i, vis)
  print('batch (before/after):')
  imshow(vis)

def plot_loss(loss_log):
  pl.figure(figsize=(10, 4))
  pl.title('Loss history (log10)')
  pl.plot(np.log10(loss_log), '.', alpha=0.1)
  pl.show()


In [ ]:
# @title Eval function


def eval_perform_steps(ca, x, yt, num_steps):
  yt_label = tf.argmax(yt, axis=-1)

  live_mask = x[..., 0] > 0.1
  live_mask_fl = tf.expand_dims(tf.cast(live_mask, tf.float32), -1)
  dead_channel = tf.cast(x[..., :1] <= 0.1, tf.float32)

  # for now the metric is aggregating everything.
  total_count = tf.reduce_sum(tf.cast(live_mask, tf.float32))

  avg_accuracy_list = []
  avg_total_agreement_list = []
  for _ in range(1, num_steps + 1):
    x = ca(x)

    y = ca.classify(x)
    y_label = tf.argmax(y, axis=-1)

    correct = tf.equal(y_label,  yt_label) & live_mask
    total_correct = tf.reduce_sum(tf.cast(correct, tf.float32))
    avg_accuracy_list.append((total_correct/total_count * 100).numpy().item())

    # agreement metrics
    # Important to exclude dead cells:
    y = y * live_mask_fl
    y_label_plus_mask = tf.argmax(tf.concat([y, dead_channel], -1), axis=-1)
    all_counts = []
    for idx in range(10):
      count_i = tf.reduce_sum(
          tf.cast(tf.equal(y_label_plus_mask, idx), tf.int32), axis=[1,2])
      all_counts.append(count_i)
    all_counts_t = tf.stack(all_counts, 1)
    # Now the trick is that if there is a total agreement, their sum is the same
    # as their max.
    equality = tf.equal(tf.reduce_max(all_counts_t, axis=1),
                        tf.reduce_sum(all_counts_t, axis=1))
    sum_agreement = tf.reduce_sum(tf.cast(equality, tf.float32))
    avg_total_agreement_list.append(sum_agreement.numpy().item() / y.shape[0] * 100)

  return avg_accuracy_list, avg_total_agreement_list

def eval_batch_fn(ca, x_test, y_test_pic, num_steps, mutate):
  x = ca.initialize(x_test)
  yt = y_test_pic

  avg_acc_l_1, avg_tot_agr_l_1 = eval_perform_steps(ca, x, yt, num_steps)
  if not mutate:
    return avg_acc_l_1, avg_tot_agr_l_1
  # Accuracy after mutation!
  new_idx = np.random.randint(0, x_test.shape[0]-1, size=x_test.shape[0])
  new_x, yt = x_test[new_idx], y_test_pic[new_idx]
  new_x = tf.reshape(new_x, [-1, 28, 28, 1])
  mutate_mask = tf.cast(new_x > 0.1, tf.float32)

  x = tf.concat([new_x, x[:,:,:,1:] * mutate_mask], -1)

  avg_acc_l_2, avg_tot_agr_l_2 = eval_perform_steps(ca, x, yt, num_steps)

  return avg_acc_l_1 + avg_acc_l_2, avg_tot_agr_l_1 + avg_tot_agr_l_2

def eval_all(ca, num_steps, mutate):
  all_accuracies = []
  all_agreements = []

  # total test set is 10000
  num_batches = 10
  eval_bs = 10000 // num_batches
  for i in range(num_batches):
    print("On batch {}".format(i))
    x_set = x_test[eval_bs*i:eval_bs*(i+1)]
    y_set = y_test_pic[eval_bs*i:eval_bs*(i+1)]
    acc_i, agr_i = eval_batch_fn(ca, x_set, y_set, num_steps, mutate)
    all_accuracies.append(acc_i)
    all_agreements.append(agr_i)

  all_accuracies = [sum(l)/num_batches for l in zip(*all_accuracies)]
  all_agreements = [sum(l)/num_batches for l in zip(*all_agreements)]
  return all_accuracies, all_agreements


# Training

In [ ]:
# Init training

ca = CAModel()


def individual_l2_loss(x, y):
  t = y - ca.classify(x)
  return tf.reduce_sum(t**2, [1, 2, 3]) / 2

def batch_l2_loss(x, y):
  return tf.reduce_mean(individual_l2_loss(x, y))

def batch_ce_loss(x, y):
  one_hot = tf.argmax(y, axis=-1)
  # It's ok even if the loss is computed on "dead" cells. Anyway they shouldn't
  # get any gradient propagated through there.
  return tf.compat.v1.losses.sparse_softmax_cross_entropy(one_hot, x)

assert LOSS_TYPE in ["l2", "ce"]
loss_fn = batch_l2_loss if LOSS_TYPE == "l2" else batch_ce_loss


loss_log = []

lr = 1e-3
lr_sched = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
      [30000, 70000], [lr, lr*0.1, lr*0.01])
trainer = tf.keras.optimizers.Adam(lr_sched)

starting_indexes = np.random.randint(0, x_train.shape[0]-1, size=POOL_SIZE)
pool = SamplePool(x=ca.initialize(x_train[starting_indexes]).numpy(), y=y_train_pic[starting_indexes])

!mkdir -p train_log && rm -f train_log/*

In [ ]:
# Training loop

@tf.function
def train_step(x, y):
  iter_n = 20
  with tf.GradientTape() as g:
    for i in tf.range(iter_n):
      x = ca(x)
    loss = batch_l2_loss(x, y)
  grads = g.gradient(loss, ca.weights)
  grads = [g/(tf.norm(g)+1e-8) for g in grads]
  trainer.apply_gradients(zip(grads, ca.weights))
  return x, loss

for i in range(1, 100000+1):
  if USE_PATTERN_POOL:
    batch = pool.sample(BATCH_SIZE)
    x0 = np.copy(batch.x)
    y0 = batch.y
    # we want half of them new. We remove 1/4 from the top and 1/4 from the
    # bottom.
    q_bs = BATCH_SIZE // 4

    new_idx = np.random.randint(0, x_train.shape[0]-1, size=q_bs)
    x0[:q_bs] = ca.initialize(x_train[new_idx])
    y0[:q_bs] = y_train_pic[new_idx]

    new_idx = np.random.randint(0, x_train.shape[0]-1, size=q_bs)
    new_x, new_y = x_train[new_idx], y_train_pic[new_idx]
    if MUTATE_POOL:
      new_x = tf.reshape(new_x, [q_bs, 28, 28, 1])
      mutate_mask = tf.cast(new_x > 0.1, tf.float32)
      mutated_x = tf.concat([new_x, x0[-q_bs:,:,:,1:] * mutate_mask], -1)

      x0[-q_bs:] = mutated_x
      y0[-q_bs:] = new_y

    else:
      x0[-q_bs:] = ca.initialize(new_x)
      y0[-q_bs:] = new_y

  else:
    b_idx = np.random.randint(0, x_train.shape[0]-1, size=BATCH_SIZE)
    x0 = ca.initialize(x_train[b_idx])
    y0 = y_train_pic[b_idx]

  x, loss = train_step(x0, y0)

  if USE_PATTERN_POOL:
    batch.x[:] = x
    batch.y[:] = y0 # This gets reordered, so you need to change it.
    batch.commit()

  step_i = len(loss_log)
  loss_log.append(loss.numpy())

  if step_i%100 == 0:
    generate_pool_figures(ca, pool, step_i)
  if step_i%200 == 0:
    clear_output()
    visualize_batch(ca, x0, x, step_i)
    plot_loss(loss_log)
  if step_i%10000 == 0:
    export_model(ca, 'train_log/%07d'%step_i)

  print('\r step: %d, log10(loss): %.3f'%(len(loss_log), np.log10(loss)), end='')

In [ ]:
# useful code if you end up interrupting the run.
print(step_i)
export_model(ca, 'train_log/%07d'%step_i)

In [ ]:
# @title eval metrics

eval_batch = 1000
num_iters = 10

all_accuracies, all_agreements = eval_all(ca, num_steps=200, mutate=True)

pl.figure(figsize=(10, 4))
pl.title('Average cell accuracy over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average cell accuracy (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_accuracies, label="ca")
#pl.plot(l2_all_accuracies, label=label_2)
pl.legend()
pl.show()

pl.figure(figsize=(10, 4))
pl.title('Average total agreement across batch over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average total agreement (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_agreements, label="ca")
#pl.plot(l2_all_accuracies, label=label_2)
pl.legend()
pl.show()

for idx in [0, 25, 50, 99, 100, 125, 150, 199]:
  print(idx, all_agreements[idx])
    

In [ ]:
#@title training progress (batches)
frames = sorted(glob.glob('train_log/batches_*.jpg'))
mvp.ImageSequenceClip(frames, fps=10.0).write_videofile('batches.mp4')
mvp.ipython_display('batches.mp4')

In [ ]:
#@title pool contents
#!rm -f train_log/*_pool.jpg

frames = sorted(glob.glob('train_log/*_pool.jpg'))[:80]
mvp.ImageSequenceClip(frames, fps=5.0).write_videofile('pool.mp4')
mvp.ipython_display('pool.mp4')

# pretrained models

Please run the cell below to download pretrained models that are used to generate the subsequent figures. They are also used in the demo section.

In [ ]:
!wget -O models.zip 'https://github.com/google-research/self-organising-systems/blob/master/assets/mnist_ca/models.zip?raw=true'
!unzip -oq models.zip

def get_exp_path(
    prefix, use_sample_pool, mutate_pool, loss_type, add_noise):
  path = prefix
  path += 'use_sample_pool_%r mutate_pool_%r '%(use_sample_pool, mutate_pool)
  path += 'loss_type_%s '%(loss_type)
  path += 'add_noise_%r'%(add_noise)
  path += '/0100000'
  return path

def get_model(use_sample_pool=True, mutate_pool=True, loss_type="l2", add_noise=True,
              prefix="models/", output='model'):
  path = get_exp_path(
      prefix, use_sample_pool, mutate_pool, loss_type, add_noise)
  assert output in ['model', 'json']
  if output == 'model':
    ca = CAModel(add_noise=add_noise)
    ca.load_weights(path)
    return ca
  elif output == 'json':
    return open(path+'.json', 'r').read()

# Figures

In [ ]:
# @title mosaic of pictures

num_digits = 144

indexes = np.random.randint(0, x_train.shape[0]-1, size=num_digits)
figures = color_labels(x_train[indexes], y_train_pic[indexes], dtype=tf.float32)
tile_digits = generate_tiled_figures(figures, fade_by=0.1)

imshow(zoom(tile_digits))



## visualize runs

In [ ]:
def make_run_videos(ca, num_steps, eval_bs, prefix, disable_black=False):
  new_idx = np.random.randint(0, x_train.shape[0]-1, size=eval_bs)
  x = ca.initialize(x_train[new_idx])

  frames = []

  fpath = 'figures_log/{}_{:04d}.png'.format(prefix, 0)
  # Always show black on the first frame.
  imwrite(fpath, tile2d(classify_and_color(ca, x, disable_black=False)))
  frames.append(fpath)


  for i in range(1, num_steps + 1):
    x = ca(x)

    fpath = 'figures_log/{}_{:04d}.png'.format(prefix, i)
    imwrite(fpath, tile2d(classify_and_color(ca, x, disable_black=disable_black)))
    frames.append(fpath)

  # then mutate
  new_idx = np.random.randint(0, x_train.shape[0]-1, size=eval_bs)
  new_x = x_train[new_idx]
  new_x = tf.reshape(new_x, [eval_bs, 28, 28, 1])
  mutate_mask = tf.cast(new_x > 0.1, tf.float32)
  x = tf.concat([new_x, x[:,:,:,1:] * mutate_mask], -1)


  for i in range(num_steps + 1,num_steps*2 + 1):
    x = ca(x)

    #colored_y = classify_and_color(ca, x)

    fpath = 'figures_log/{}_{:04d}.png'.format(prefix, i)
    imwrite(fpath, tile2d(classify_and_color(ca, x, disable_black=disable_black)))
    frames.append(fpath)


  #frames = sorted(glob.glob('figures_log/ce_internal_states_*.png'))
  mvp.ImageSequenceClip(frames, fps=40.0).write_videofile(prefix + '.mp4')
  mvp.ImageSequenceClip(frames[::40], fps=5.0).write_gif(prefix+ '.gif')


In [ ]:
# @title visualize CE runs
! mkdir figures_log && rm -f figures_log/ce_runs*

eval_bs= 100
num_steps = 400

ca = get_model(add_noise=False, loss_type='ce')

make_run_videos(ca, num_steps, eval_bs, "ce_runs", disable_black=True)

mvp.ipython_display('ce_runs.mp4')

In [ ]:
from google.colab import files
files.download('ce_runs.mp4')
files.download('ce_runs.gif')


In [ ]:
# @title visualize L2 runs
! mkdir figures_log && rm -f figures_log/ce_runs*

eval_bs= 100
num_steps = 400

ca = get_model(add_noise=True, loss_type='l2')

make_run_videos(ca, num_steps, eval_bs, "l2_runs")

mvp.ipython_display('l2_runs.mp4')

In [ ]:
from google.colab import files
files.download('l2_runs.mp4')
files.download('l2_runs.gif')


In [ ]:
# @title visualize L2 runs quant
! mkdir figures_log && rm -f figures_log/ce_runs*

eval_bs= 100
num_steps = 400

ca = get_model(add_noise=True, loss_type='l2', quant_states=True)

make_run_videos(ca, num_steps, eval_bs, "l2_q_runs")

mvp.ipython_display('l2_q_runs.mp4')

In [ ]:
# @title CE eval for experiment 1
# @title comparing

ca = get_model(add_noise=False, loss_type='ce')
label_1 = 'Cross Entropy'

all_accuracies, all_agreements = eval_all(ca, num_steps=200, mutate=True)

for idx in [0, 25, 50, 99, 100, 125, 150, 199, 200, 225, 250, 299, 300, 325, 350, 399]:
  print("Step ", idx,"accuracy", all_accuracies[idx],"agreement", all_agreements[idx])


In [ ]:
# plot them.
pl.figure(figsize=(10, 4))

#pl.subplot(211)
pl.title('Average cell accuracy over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average cell accuracy (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_accuracies, label=label_1)
#pl.plot(l2_all_accuracies, label=label_2)
pl.legend()
pl.show()

#pl.subplot(212)
pl.figure(figsize=(10, 4))
pl.title('Average total agreement across batch over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average total agreement (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_agreements, label=label_1)
#pl.plot(l2_all_accuracies, label=label_2)
pl.legend()
pl.show()

In [ ]:
# @title comparing CE, L2, L2+noise

ca = get_model(add_noise=False, loss_type='ce')
all_accuracies_1, all_agreements_1 = eval_all(ca, num_steps=200, mutate=True)

print("CE")
for idx in [0, 25, 50, 99, 100, 125, 150, 199, 200, 225, 250, 299, 300, 325, 350, 399]:
  print("Step ", idx,"accuracy", all_accuracies_1[idx],"agreement", all_agreements_1[idx])

ca = get_model(add_noise=False, loss_type='l2')
all_accuracies_2, all_agreements_2 = eval_all(ca, num_steps=200, mutate=True)

print("L2")
for idx in [0, 25, 50, 99, 100, 125, 150, 199, 200, 225, 250, 299, 300, 325, 350, 399]:
  print("Step ", idx,"accuracy", all_accuracies_2[idx],"agreement", all_agreements_2[idx])

ca = get_model(add_noise=True, loss_type='l2')
all_accuracies_3, all_agreements_3 = eval_all(ca, num_steps=200, mutate=True)

for idx in [0, 25, 50, 99, 100, 125, 150, 199, 200, 225, 250, 299, 300, 325, 350, 399]:
  print("Step ", idx,"accuracy", all_accuracies_3[idx],"agreement", all_agreements_3[idx])


In [ ]:
def get_highest_point(arr):
  highest_i = 0
  highest_val = 0
  for i in range(len(arr)):
    if arr[i] > highest_val:
      highest_i = i
      highest_val = arr[i]
  return highest_i, highest_val

highest_i, highest_val = get_highest_point(all_accuracies_1[:200])
print("CE top accuracy:", highest_i, highest_val)
highest_i, highest_val = get_highest_point(all_agreements_1[:200])
print("CE top agreement:", highest_i, highest_val)

highest_i, highest_val = get_highest_point(all_accuracies_2[:200])
print("L2 top accuracy:", highest_i, highest_val)
highest_i, highest_val = get_highest_point(all_agreements_2[:200])
print("L2 top agreement:", highest_i, highest_val)


highest_i, highest_val = get_highest_point(all_accuracies_3[:200])
print("L2 + noise top accuracy:", highest_i, highest_val)
highest_i, highest_val = get_highest_point(all_agreements_3[:200])
print("L2 + noise top agreement:", highest_i, highest_val)

In [ ]:

label_1 = 'Cross Entropy'
label_2 = 'L2'
label_3 = 'L2 + Residual Noise'

pl.figure(figsize=(15, 10))
pl.subplot(211)
pl.title('Average cell accuracy over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average cell accuracy (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_accuracies_1, label=label_1)
pl.plot(all_accuracies_2, label=label_2)
pl.plot(all_accuracies_3, label=label_3)
pl.legend()
#pl.show()


#pl.figure(figsize=(10, 4))
pl.subplot(212)
pl.title('Average total agreement across batch over steps (%)')
pl.xlabel('Number of steps')
pl.ylabel('Average total agreement (%)')
pl.xlim(0, 400)
pl.ylim(0, 100)
pl.plot(all_agreements_1, label=label_1)
pl.plot(all_agreements_2, label=label_2)
pl.plot(all_agreements_3, label=label_3)
#pl.legend()
pl.show()


## Internal states



In [ ]:

from skimage.transform import resize


def vis_internal_states(x):
  vis = x.numpy().transpose([2, 0, 1])
  vis = tile2d(vis, 10)
  vis = np.clip(vis, -1., 1.)
  vis = zoom(pl.cm.RdBu_r(vis*0.4+0.5), 2)
  return vis

def vis_classifications(y):
  vis_y = y.numpy().transpose([2, 0, 1])
  vis_y = tile2d(vis_y, 10)
  vis_y = np.clip(vis_y, -1., 1.)
  vis_y = zoom(pl.cm.RdBu_r(vis_y*0.4+0.5), 2)
  return vis_y

# Expect a single instance. not a batch.
def vis_internal_states_legacy(x, y, y_colored):
  vis = extract_internal_states(x)

  current_prediction = classify_and_color(ca, tf.expand_dims(x, 0), disable_black=True)[0]
  above_part = zoom(np.hstack([current_prediction, y_colored]), 5)

  above_part = resize(above_part, (vis.shape[1] // 2, vis.shape[1]))
  #above_part = above_part.astype(np.float32) / 255.
  above_part = np.concatenate([above_part, np.ones_like(above_part[...,:1])], -1)

  vis_y = vis_classifications(y)
  h_vis_y = vis_y.shape[0] * vis.shape[1] // vis_y.shape[1]
  vis_y = resize(vis_y, (h_vis_y, vis.shape[1]))

  vis = np.vstack([above_part, vis, vis_y])

  return vis

def vis_for_video(x, y, legend=None):
  vis = vis_internal_states(x)

  current_prediction = classify_and_color(ca, tf.expand_dims(x, 0), disable_black=True)[0]
  above_part = zoom(current_prediction, 10)

  above_part = resize(above_part, (vis.shape[1], vis.shape[1]))
  # add A channel
  above_part = np.concatenate([above_part, np.ones_like(above_part[...,:1])], -1)

  vis_y = vis_classifications(y)
  h_vis_y = vis_y.shape[0] * vis.shape[1] // vis_y.shape[1]
  vis_y = resize(vis_y, (h_vis_y, vis.shape[1]))


  vis = np.vstack([above_part, vis, vis_y])
  if legend is not None:
    l_h = legend.shape[1] * vis.shape[0] // legend.shape[0]
    legend = resize(legend, (vis.shape[0], l_h))
    vis = np.hstack([vis, legend])

  return vis


In [ ]:
# @title CE internal states
!rm -f figures_log/internal_states*
!mkdir figures_log

legend = numbers_legend
# need to add the A channel.
legend = np.concatenate([legend, np.ones_like(legend[...,:1])], -1)

x_set = x_test
y_set = y_test_pic

ca = get_model(add_noise=False, loss_type='ce')

choose_at_random = True
if choose_at_random:
  b_idx = np.random.randint(0, x_set.shape[0]-1, size=1)
  print("Random idx: {}".format(b_idx))
else:
  b_idx = 8198

x0 = ca.initialize(x_train[b_idx])
y0 = color_labels(x0[..., 0], y_train_pic[b_idx])[0].numpy().astype(np.float32) / 255.

# here and below: for ce, uncomment this and use it instead of the classify.
ce_y = tf.nn.softmax(ca.classify(x0), -1)[0]

frames = []

#imwrite('figures_log/ce_internal_states_%04d.png'%0, 
#        vis_internal_states(x0[0], ce_y, y0))

fpath = 'figures_log/ce_internal_states_%04d.png'%0
imwrite(fpath, vis_for_video(x0[0], ce_y, legend))
frames.append(fpath)

#imshow(frames[-1])


for i in range(1, 401):
  if i % 50 == 0: print(i)
  x0 = ca(x0)
  ce_y = tf.nn.softmax(ca.classify(x0), -1)[0]
  fpath = 'figures_log/ce_internal_states_%04d.png'%i
  imwrite(fpath, vis_for_video(x0[0], ce_y, legend))
  frames.append(fpath)


#frames = sorted(glob.glob('figures_log/ce_internal_states_*.png'))
mvp.ImageSequenceClip(frames, fps=20.0).write_videofile('ce_internal_states.mp4')
mvp.ImageSequenceClip(frames[::20], fps=5.0).write_gif('ce_internal_states.gif')

#mvp.ipython_display('ce_internal_states.gif')
mvp.ipython_display('ce_internal_states.mp4')

In [ ]:
from google.colab import files
files.download('ce_internal_states.mp4')
files.download('ce_internal_states.gif')


In [ ]:
# @title Internal state comparison L2 vs CE
! mkdir figures_log && rm -f figures_log/comp_internal_states*

legend = numbers_legend
# need to add the A channel.
legend = np.concatenate([legend, np.ones_like(legend[...,:1])], -1)

ca_ce = get_model(add_noise=False, loss_type='ce')
ca_l2 = get_model(add_noise=True, loss_type='l2')

x_set = x_test
y_set = y_test_pic

choose_at_random = True
if choose_at_random:
  b_idx = np.random.randint(0, x_set.shape[0]-1, size=1)
  print("Random idx: {}".format(b_idx))
else:
  b_idx = 446

x0_l2 = ca_l2.initialize(x_set[b_idx])
x0_ce = ca_ce.initialize(x_set[b_idx])
y0 = color_labels(x0_l2[..., 0], y_set[b_idx])[0].numpy().astype(np.float32) / 255.

l2_vis = vis_for_video(x0_l2[0], ca_l2.classify(x0_l2)[0], legend)
ce_y = tf.nn.softmax(ca_ce.classify(x0_ce), -1)[0]
ce_vis = vis_for_video(x0_ce[0], ce_y)
vis = np.hstack([l2_vis, ce_vis])

#imwrite('figures_log/comp_internal_states_%04d.png'%0, vis)
fpath = 'figures_log/comp_internal_states_%04d.png'%0
imwrite(fpath, vis)
frames.append(fpath)

#imshow(frames[-1])

frames = []

for i in range(1, 401):
  if i % 50 == 0: print(i)
  x0_l2 = ca_l2(x0_l2)
  x0_ce = ca_ce(x0_ce)
  l2_vis = vis_for_video(x0_l2[0], ca_l2.classify(x0_l2)[0], legend)
  ce_y = tf.nn.softmax(ca_ce.classify(x0_ce), -1)[0]
  ce_vis = vis_for_video(x0_ce[0], ce_y)
  vis = np.hstack([l2_vis, ce_vis])

  fpath = 'figures_log/comp_internal_states_%04d.png'%i
  imwrite(fpath, vis)
  frames.append(fpath)
  #imwrite('figures_log/comp_internal_states_%04d.png'%i, vis)

# todo add a teaser low frame gif.

frames = sorted(glob.glob('figures_log/comp_internal_states_*.png'))

mvp.ImageSequenceClip(frames, fps=20.0).write_videofile('internal_states_comparison.mp4')
mvp.ImageSequenceClip(frames[::20], fps=5.0).write_gif('internal_states_comparison.gif')
mvp.ipython_display('internal_states_comparison.mp4')

In [ ]:
from google.colab import files
files.download('internal_states_comparison.mp4')
files.download('internal_states_comparison.gif')

In [ ]:
# @title Create the data for the distill demo

def create_numbers_legend(x_set, y_set, y_set_pic):
  result_y = []
  for i in range(10):
    for x, y, y_pic in zip(x_set, y_set, y_set_pic):
      if y == i:
        result_y.append(color_labels(x, y_pic))
        break
  assert len(result_y) == 10
  return tile2d(np.stack(result_y), 5)


def find_several_numbers(x_set, y_set, num_each):
  result = []
  for i in range(10):
    i_numbers = []
    n = 0
    for x, y in zip(x_set, y_set):
      if y == i:
        i_numbers.append(x)
        n += 1
        if n == num_each: break
    result.append(np.stack(i_numbers))
  return np.stack(result)


#numbers_legend, x_legend = find_different_numbers(x_train, y_train, y_train_pic)
demo_numbers_legend = create_numbers_legend(x_train, y_train, y_train_pic)

n = 20
demo_samples = find_several_numbers(x_train, y_train, n)
print(demo_samples.shape)
demo_samples = np.concatenate([tile2d(dsi, n) for dsi in demo_samples], 0)
print(demo_samples.shape)
#demo_samples = demo_samples.reshape([28 * 10, 28* n])



imshow(zoom(demo_numbers_legend))
im = PIL.Image.fromarray(zoom(demo_numbers_legend))
im.save("demo_numbers_legend.png")

print("Storing demo_samples for use in the demo.")
#pl.imshow(demo_samples.reshape((-1, 28)))
import json
many_samples_str = json.dumps(demo_samples.tolist())
print(len(many_samples_str))

pl.imshow(demo_samples)
demo_samples *= 255.
im = PIL.Image.fromarray(demo_samples.astype(np.uint8)).convert("L")
im.save("demo_samples.png")


In [ ]:
!ls -lh demo_numbers_legend.png
!ls -lh demo_samples.png

In [ ]:
from google.colab import files
files.download('demo_samples.png')

# Demo


In [ ]:
#@title TensorFlow.js Demo small {run:"auto", vertical-output: true}
#@markdown Select "CHECKPOINT" model to load the checkpoint created by running 
#@markdown cells from the "Training" section of this notebook.
#@markdown Technical note: CE models should be rendered differently to avoid
#@markdown black pixels showing for low magnitude.
import IPython.display

model_source = "LOAD"  #@param ['CHECKPOINT', 'LOAD']
model_type = '3 mutating'  #@param ['1 naive', '2 persistent', '3 mutating']
loss_type = "l2"  #@param ['l2', 'ce']
add_noise = "True"  #@param ['True', 'False']
#quant_states = "True"  #@param ['True', 'False']

#@markdown draw with left click, hold shift for erasing

if model_source != 'CHECKPOINT':
  use_sample_pool, mutate_pool = {
      '1 naive': (False, False),
      '2 persistent': (True, False),
      '3 mutating': (True, True)
      }[model_type]
  add_noise = add_noise == 'True'

  model_str = get_model(
      use_sample_pool=use_sample_pool, mutate_pool=mutate_pool,
      loss_type=loss_type, add_noise=add_noise,
      output='json')
else:
  last_checkpoint_fn = sorted(glob.glob('train_log/*.json'))[-1]
  model_str = open(last_checkpoint_fn).read()

data_js = '''
  window.GRAPH_URL = URL.createObjectURL(new Blob([`%s`], {type: 'application/json'}));
  window.SAMPLES = %s
'''%(model_str, samples_str)

display(IPython.display.Javascript(data_js))


IPython.display.HTML('''
<script src=\"https://unpkg.com/@tensorflow/tfjs@latest/dist/tf.min.js\"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/cash/4.1.2/cash.min.js"></script>
<div class="slidecontainer">
    brushSize:
    <input type="range" min="1" max="10" value="4" class="slider" id="brushSlider">
    <span id='radius'>2.5</span>
</div>
<canvas id='canvas' style="border: 1px solid black; image-rendering: pixelated;"></canvas>
<script>
  "use strict";

  // Adds the WASM backend to the global backend registry.
  //import '@tensorflow/tfjs-backend-wasm';
  // Set the backend to WASM and wait for the module to be ready.
const main = async () => {

  const sleep = (ms)=>new Promise(resolve => setTimeout(resolve, ms));
  
  const parseConsts = model_graph=>{
    const dtypes = {'DT_INT32':['int32', 'intVal', Int32Array],
                    'DT_FLOAT':['float32', 'floatVal', Float32Array]};
    
    const consts = {};
    model_graph.modelTopology.node.filter(n=>n.op=='Const').forEach((node=>{
      const v = node.attr.value.tensor;
      const [dtype, field, arrayType] = dtypes[v.dtype];
      if (!v.tensorShape.dim) {
        consts[node.name] = [tf.scalar(v[field][0], dtype)];
      } else {
        const shape = v.tensorShape.dim.map(d=>parseInt(d.size));
        let arr;
        if (v.tensorContent) {
          const data = atob(v.tensorContent);
          const buf = new Uint8Array(data.length);
          for (var i=0; i<data.length; ++i) {
            buf[i] = data.charCodeAt(i);
          }
          arr = new arrayType(buf.buffer);
        } else {
          const size = shape.reduce((a, b)=>a*b);
          arr = new arrayType(size);
          arr.fill(v[field][0]);
        }
        consts[node.name] = [tf.tensor(arr, shape, dtype)];
      }
    }));
    return consts;
  }
  
  let paused = false;
  let visibleChannel = -1;
  let firingChance = 0.5;
  let drawRadius = 2.5;

  $('#brushSlider').on('input', e=>{
      drawRadius = parseFloat(e.target.value)/2.0;
      $('#radius').text(drawRadius);
  });

  const colorLookup = tf.tensor([
      [128, 0, 0],
      [230, 25, 75],
      [70, 240, 240],
      [210, 245, 60],
      [250, 190, 190],
      [170, 110, 40],
      [170, 255, 195],
      [165, 163, 159],
      [0, 128, 128],
      [128, 128, 0],
      [0, 0, 0], // This is the default for digits.
      [255, 255, 255] // This is the background.
      ])

  let backgroundWhite = true;


  const run = async () => {
      const r = await fetch(GRAPH_URL);
      const consts = parseConsts(await r.json());

      //const samples = tf.tensor(SAMPLES);
      //console.log(samples);

      const model = await tf.loadGraphModel(GRAPH_URL);

      const samples = tf.tensor(SAMPLES);
      console.log(samples.shape);
      //const samples = tf.zeros([2,5, 28, 28]);

      console.log("Loaded model")
      Object.assign(model.weights, consts);
      // samples.gather(tf.range(0, 4, 1, 'int32')
      const D = 28 * 2;
      const state = tf.variable(tf.zeros([1, D, D, 20]));
      const [_, h, w, ch] = state.shape;

      const scale = 8;

      const canvas = document.getElementById('canvas');
      const ctx = canvas.getContext('2d');
      canvas.width = w * scale;
      canvas.height = h * scale;

      const drawing_canvas = new OffscreenCanvas(w, h);
      const draw_ctx = drawing_canvas.getContext('2d');

      // Useful for understanding background color.
      
      //let blackAndWhite = tf.zeros();//.fill(0.01);
      let arr = new Float32Array(h * w * 2);
      arr.fill(0.01);
      const blackAndWhiteFull = tf.tensor(arr, [1,h,w,2], tf.float32)

      const drawCanvas = (imgd, e) => {
          var matrix = [];
          for(let i=0; i<imgd.width; i++) {
              matrix[i] = [];
              for(let j=0; j<imgd.height; j++) {
                  let intensity = imgd.data[(imgd.height*j*4 + i*4)];
                  // For drawing, we want to add shades of grey. For erasing, we don't.
                  if (!e.shiftKey) {
                    intensity *= (imgd.data[(imgd.height*j*4 + i*4 + 3)] / 255);
                  }
                  matrix[i][j] = intensity;
              }
          }

          tf.tidy(() => {
              const stroke = tf.tensor(matrix).transpose().toFloat().div(255.).expandDims(0).expandDims(3);
              const stroke_pad = tf.concat([stroke, tf.zeros([1, h, w, ch-1])], 3);
              const mask = tf.tensor(1.).sub(stroke);
              if (e.shiftKey) {
                  state.assign(state.mul(mask));
              } else {
                  state.assign(state.mul(mask).add(stroke_pad));
              }
          });

          // Then clear the canvas.
          draw_ctx.clearRect(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
      }

      const line = (x0, y0, x1, y1, r, e) => {
          draw_ctx.beginPath();
          draw_ctx.moveTo(x0, y0);
          draw_ctx.lineTo(x1, y1);
          draw_ctx.strokeStyle = "#ff0000";
          // Erasing has a much larger radius.
          draw_ctx.lineWidth = (e.shiftKey ? 5. * r : r);
          draw_ctx.stroke();

          const imgd = draw_ctx.getImageData(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
          drawCanvas(imgd, e);
      }


      const circle = (x, y, r, e) => {
          draw_ctx.beginPath();

          const drawRadius = (e.shiftKey ? 5. * r : r) / 3.;

          draw_ctx.arc(x, y, drawRadius, 0, 2 * Math.PI, false);
          draw_ctx.fillStyle = "#ff0000";
          draw_ctx.fill();
          draw_ctx.lineWidth = 1;
          draw_ctx.strokeStyle = "#ff0000";
          draw_ctx.stroke();

          const imgd = draw_ctx.getImageData(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
          drawCanvas(imgd, e);
      }

      const draw_r = 2.0;


      const getClickPos = e=>{
          const x = Math.floor((e.pageX-e.target.offsetLeft) / scale);
          const y = Math.floor((e.pageY-e.target.offsetTop) / scale);
          return [x, y];
      }

      let lastX = 0;
      let lastY = 0;

      canvas.onmousedown = e => {
          const [x, y] = getClickPos(e);
          lastX = x;
          lastY = y;
          circle(x,y,drawRadius, e);
      }
      canvas.onmousemove = e => {
          const [x, y] = getClickPos(e);
          if (e.buttons == 1) {
              line(lastX,lastY, x,y,drawRadius, e);
          }
          lastX = x;
          lastY = y;
      }
      const render = async () => {
        if (!paused) {
          tf.tidy(() => {
              state.assign(model.execute(
                { x: state,
                  fire_rate: tf.tensor(firingChance),
                  manual_noise: tf.randomNormal([1, h, w, ch-1], 0., 0.02)},
                ['Identity']));
          });
        }
        const imageData = tf.tidy(() => {
            let rgbaBytes;
            let rgba;
            if (visibleChannel < 0) {
                const isGray = state.slice([0,0,0,0],[1, h, w, 1]).greater(0.1).toFloat();
                const isNotGray = tf.tensor(1.).sub(isGray);

                const bnwOrder = backgroundWhite ?  [isGray, isNotGray] : [isNotGray, isGray];
                let blackAndWhite = blackAndWhiteFull.mul(tf.concat(bnwOrder, 3));

                const grey = state.gather([0], 3).mul(255);
                const rgb = tf.gather(colorLookup,
                                      tf.argMax(
                                      tf.concat([
                  state.slice([0,0,0,ch-10],[1,h,w,10]),
                  blackAndWhite], 3), 3));

                rgba = tf.concat([rgb, grey], 3)
            } else {
                rgba = state.gather([visibleChannel, visibleChannel, visibleChannel], 3)
                  .pad([[0, 0], [0, 0], [0, 0], [0, 1]], 1).mul(255);
            }
            rgbaBytes = new Uint8ClampedArray(rgba.dataSync());

            return new ImageData(rgbaBytes, w, h);
        });
        const image = await createImageBitmap(imageData);
        //ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.fillStyle = backgroundWhite ? "#ffffff" : "#000000";
        ctx.fillRect(0, 0, canvas.width, canvas.height);
        ctx.imageSmoothingEnabled = false;
        ctx.drawImage(image, 0, 0, canvas.width, canvas.height);

        requestAnimationFrame(render);
      }
      render();
  }

  run();
}
main();
  //tf.setBackend('wasm').then(() => main());

  
</script>
''')

In [ ]:
#@title TensorFlow.js Demo Full whiteboard {run:"auto", vertical-output: true}
#@markdown Select "CHECKPOINT" model to load the checkpoint created by running 
#@markdown cells from the "Training" section of this notebook.
#@markdown Technical note: CE models should be rendered differently to avoid
#@markdown black pixels showing for low magnitude.
import IPython.display

model_source = "LOAD"  #@param ['CHECKPOINT', 'LOAD']
model_type = '3 mutating'  #@param ['1 naive', '2 persistent', '3 mutating']
loss_type = "l2"  #@param ['l2', 'ce']
add_noise = "True"  #@param ['True', 'False']
#quant_states = "True"  #@param ['True', 'False']

#@markdown draw with left click, hold shift for erasing

if model_source != 'CHECKPOINT':
  use_sample_pool, mutate_pool = {
      '1 naive': (False, False),
      '2 persistent': (True, False),
      '3 mutating': (True, True)
      }[model_type]
  add_noise = add_noise == 'True'

  model_str = get_model(
      use_sample_pool=use_sample_pool, mutate_pool=mutate_pool,
      loss_type=loss_type, add_noise=add_noise,
      output='json')
else:
  last_checkpoint_fn = sorted(glob.glob('train_log/*.json'))[-1]
  model_str = open(last_checkpoint_fn).read()

data_js = '''
  window.GRAPH_URL = URL.createObjectURL(new Blob([`%s`], {type: 'application/json'}));
  window.SAMPLES = %s
'''%(model_str, samples_str)

display(IPython.display.Javascript(data_js))


IPython.display.HTML('''
<script src=\"https://unpkg.com/@tensorflow/tfjs@latest/dist/tf.min.js\"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/cash/4.1.2/cash.min.js"></script>
<div class="slidecontainer">
    brushSize:
    <input type="range" min="1" max="10" value="4" class="slider" id="brushSlider">
    <span id='radius'>2.5</span>
</div>
<canvas id='canvas' style="border: 1px solid black; image-rendering: pixelated;"></canvas>
<script>
  "use strict";

  // Adds the WASM backend to the global backend registry.
  //import '@tensorflow/tfjs-backend-wasm';
  // Set the backend to WASM and wait for the module to be ready.
const main = async () => {

  const sleep = (ms)=>new Promise(resolve => setTimeout(resolve, ms));
  
  const parseConsts = model_graph=>{
    const dtypes = {'DT_INT32':['int32', 'intVal', Int32Array],
                    'DT_FLOAT':['float32', 'floatVal', Float32Array]};
    
    const consts = {};
    model_graph.modelTopology.node.filter(n=>n.op=='Const').forEach((node=>{
      const v = node.attr.value.tensor;
      const [dtype, field, arrayType] = dtypes[v.dtype];
      if (!v.tensorShape.dim) {
        consts[node.name] = [tf.scalar(v[field][0], dtype)];
      } else {
        const shape = v.tensorShape.dim.map(d=>parseInt(d.size));
        let arr;
        if (v.tensorContent) {
          const data = atob(v.tensorContent);
          const buf = new Uint8Array(data.length);
          for (var i=0; i<data.length; ++i) {
            buf[i] = data.charCodeAt(i);
          }
          arr = new arrayType(buf.buffer);
        } else {
          const size = shape.reduce((a, b)=>a*b);
          arr = new arrayType(size);
          arr.fill(v[field][0]);
        }
        consts[node.name] = [tf.tensor(arr, shape, dtype)];
      }
    }));
    return consts;
  }
  
  let paused = false;
  let visibleChannel = -1;
  let firingChance = 0.5;
  let drawRadius = 2.5;

  $('#brushSlider').on('input', e=>{
      drawRadius = parseFloat(e.target.value)/2.0;
      $('#radius').text(drawRadius);
  });

  const colorLookup = tf.tensor([
      [128, 0, 0],
      [230, 25, 75],
      [70, 240, 240],
      [210, 245, 60],
      [250, 190, 190],
      [170, 110, 40],
      [170, 255, 195],
      [165, 163, 159],
      [0, 128, 128],
      [128, 128, 0],
      [0, 0, 0], // This is the default for digits.
      [255, 255, 255] // This is the background.
      ])

  let backgroundWhite = true;


  const run = async () => {
      const r = await fetch(GRAPH_URL);
      const consts = parseConsts(await r.json());

      //const samples = tf.tensor(SAMPLES);
      //console.log(samples);

      const model = await tf.loadGraphModel(GRAPH_URL);

      const samples = tf.tensor(SAMPLES);
      //const samples = tf.zeros([2,5, 28, 28]);

      console.log("Loaded model")
      Object.assign(model.weights, consts);
      // samples.gather(tf.range(0, 4, 1, 'int32')
      const D = 28 * 5;
      const flatState = tf.concat([
        samples.reshape([2,5, 28, 28])
          .transpose([0,2,1,3]).reshape([1, 2 * 28, D]),
        tf.zeros([1, D-(28*2), D])],1);
      console.log(flatState)
      const state = tf.variable(
        tf.concat([flatState.expandDims(3),
                    tf.zeros([1, D, D, 19])], 3));
      const [_, h, w, ch] = state.shape;

      const scale = 8;

      const canvas = document.getElementById('canvas');
      const ctx = canvas.getContext('2d');
      canvas.width = w * scale;
      canvas.height = h * scale;

      const drawing_canvas = new OffscreenCanvas(w, h);
      const draw_ctx = drawing_canvas.getContext('2d');

      // Useful for understanding background color.
      
      //let blackAndWhite = tf.zeros();//.fill(0.01);
      let arr = new Float32Array(h * w * 2);
      arr.fill(0.01);
      const blackAndWhiteFull = tf.tensor(arr, [1,h,w,2], tf.float32)

      const drawCanvas = (imgd, e) => {
          var matrix = [];
          for(let i=0; i<imgd.width; i++) {
              matrix[i] = [];
              for(let j=0; j<imgd.height; j++) {
                  let intensity = imgd.data[(imgd.height*j*4 + i*4)];
                  // For drawing, we want to add shades of grey. For erasing, we don't.
                  if (!e.shiftKey) {
                    intensity *= (imgd.data[(imgd.height*j*4 + i*4 + 3)] / 255);
                  }
                  matrix[i][j] = intensity;
              }
          }

          tf.tidy(() => {
              const stroke = tf.tensor(matrix).transpose().toFloat().div(255.).expandDims(0).expandDims(3);
              const stroke_pad = tf.concat([stroke, tf.zeros([1, h, w, ch-1])], 3);
              const mask = tf.tensor(1.).sub(stroke);
              if (e.shiftKey) {
                  state.assign(state.mul(mask));
              } else {
                  state.assign(state.mul(mask).add(stroke_pad));
              }
          });

          // Then clear the canvas.
          draw_ctx.clearRect(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
      }

      const line = (x0, y0, x1, y1, r, e) => {
          draw_ctx.beginPath();
          draw_ctx.moveTo(x0, y0);
          draw_ctx.lineTo(x1, y1);
          draw_ctx.strokeStyle = "#ff0000";
          // Erasing has a much larger radius.
          draw_ctx.lineWidth = (e.shiftKey ? 5. * r : r);
          draw_ctx.stroke();

          const imgd = draw_ctx.getImageData(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
          drawCanvas(imgd, e);
      }


      const circle = (x, y, r, e) => {
          draw_ctx.beginPath();

          const drawRadius = (e.shiftKey ? 5. * r : r) / 3.;

          draw_ctx.arc(x, y, drawRadius, 0, 2 * Math.PI, false);
          draw_ctx.fillStyle = "#ff0000";
          draw_ctx.fill();
          draw_ctx.lineWidth = 1;
          draw_ctx.strokeStyle = "#ff0000";
          draw_ctx.stroke();

          const imgd = draw_ctx.getImageData(0, 0, draw_ctx.canvas.width, draw_ctx.canvas.height);
          drawCanvas(imgd, e);
      }

      const draw_r = 2.0;


      const getClickPos = e=>{
          const x = Math.floor((e.pageX-e.target.offsetLeft) / scale);
          const y = Math.floor((e.pageY-e.target.offsetTop) / scale);
          return [x, y];
      }

      let lastX = 0;
      let lastY = 0;

      canvas.onmousedown = e => {
          const [x, y] = getClickPos(e);
          lastX = x;
          lastY = y;
          circle(x,y,drawRadius, e);
      }
      canvas.onmousemove = e => {
          const [x, y] = getClickPos(e);
          if (e.buttons == 1) {
              line(lastX,lastY, x,y,drawRadius, e);
          }
          lastX = x;
          lastY = y;
      }
      const render = async () => {
        if (!paused) {
          tf.tidy(() => {
              state.assign(model.execute(
                { x: state,
                  fire_rate: tf.tensor(firingChance),
                  manual_noise: tf.randomNormal([1, h, w, ch-1], 0., 0.02)},
                ['Identity']));
          });
        }
        const imageData = tf.tidy(() => {
            let rgbaBytes;
            let rgba;
            if (visibleChannel < 0) {
                const isGray = state.slice([0,0,0,0],[1, h, w, 1]).greater(0.1).toFloat();
                const isNotGray = tf.tensor(1.).sub(isGray);

                const bnwOrder = backgroundWhite ?  [isGray, isNotGray] : [isNotGray, isGray];
                let blackAndWhite = blackAndWhiteFull.mul(tf.concat(bnwOrder, 3));

                const grey = state.gather([0], 3).mul(255);
                const rgb = tf.gather(colorLookup,
                                      tf.argMax(
                                      tf.concat([
                  state.slice([0,0,0,ch-10],[1,h,w,10]),
                  blackAndWhite], 3), 3));

                rgba = tf.concat([rgb, grey], 3)
            } else {
                rgba = state.gather([visibleChannel, visibleChannel, visibleChannel], 3)
                  .pad([[0, 0], [0, 0], [0, 0], [0, 1]], 1).mul(255);
            }
            rgbaBytes = new Uint8ClampedArray(rgba.dataSync());

            return new ImageData(rgbaBytes, w, h);
        });
        const image = await createImageBitmap(imageData);
        //ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.fillStyle = backgroundWhite ? "#ffffff" : "#000000";
        ctx.fillRect(0, 0, canvas.width, canvas.height);
        ctx.imageSmoothingEnabled = false;
        ctx.drawImage(image, 0, 0, canvas.width, canvas.height);

        requestAnimationFrame(render);
      }
      render();
  }

  run();
}
main();
  //tf.setBackend('wasm').then(() => main());

  
</script>
''')